<a href="https://colab.research.google.com/github/codeREXus/langchain-learnings/blob/main/Langchain_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Retreivers**

In [1]:
!pip install -U langchain==0.2.14 langchain-community langchain-core langchain-text-splitters langchain-google-genai| tail -n 1
!pip install "chromadb==0.4.24" | tail -n 1
!pip install "pypdf==4.3.1" | tail -n 1
!pip install "lark==1.1.9" | tail -n 1
!pip install 'posthog<6.0.0' | tail -n 1
!pip install 'numpy ==1.26.4' |tail -n 1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 6.33.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-proto==1.37.0, but you have opentelemetry-proto 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-sdk~=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.0 which is inc

In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/MZ9z1lm-Ui3YBp3SYWLTAQ/companypolicies.txt"

--2025-11-08 14:01:58--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/MZ9z1lm-Ui3YBp3SYWLTAQ/companypolicies.txt
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15660 (15K) [text/plain]
Saving to: ‘companypolicies.txt.1’

companypolicies.txt 100%[===================>]  15.29K  --.-KB/s    in 0s      

2025-11-08 14:01:59 (238 MB/s) - ‘companypolicies.txt.1’ saved [15660/15660]



In [3]:
!pip install sentence-transformers |tail -n 1


In [4]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.colab import userdata
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings


In [6]:
def embed_model():
  embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
  return embed_model

In [7]:
def text_splitter( data, chunk_size , chunk_overlap):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = chunk_size,
      chunk_overlap = chunk_overlap,
      length_function = len
  )
  chunks = text_splitter.split_documents(data)
  return chunks

In [20]:
def llm():
  llm = GoogleGenerativeAI(
    model='gemini-1.5-flash-latest',
    google_api_key=userdata.get('google-api-key')
  )
  return llm

In [9]:
loader = TextLoader("companypolicies.txt")
text = loader.load()
text_chunk = text_splitter(text, 200, 20)

# **Vector backed retreiver**

In [21]:
from langchain_community.vectorstores import Chroma
vectordb = Chroma.from_documents(text_chunk, embed_model())

In [11]:
query = input("input query :")
retreiver = vectordb.as_retriever()

input query :smoke


In [12]:
docs = retreiver.invoke(query)
for doc in docs:
  print(doc.page_content)

smoke and to maintain the overall cleanliness of the premises.
5.	Smoking Policy
We appreciate your cooperation in maintaining a smoke-free and safe environment for all.
Designated Smoking Areas: Smoking is only permitted in designated smoking areas, as marked by appropriate signage. These areas have been chosen to minimize exposure to secondhand smoke and to


# **MMR SEARCH**
this is a technique used to balance relevance and diversity of results. This helps in selecting a diverse set of relevant docs.

In [13]:
retriever = vectordb.as_retriever(search_type="mmr")
docs = retriever.invoke(query)


In [14]:
for doc in docs:
  print(doc.page_content)

smoke and to maintain the overall cleanliness of the premises.
5.	Smoking Policy
those seeking treatment.
or suspicious activities related to your mobile device.


# **Multi Query retreiver**
 using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and then takes the unique union of these results to form a larger set of potentially relevant documents.

In [15]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ioch1wsxkfqgfLLgmd-6Rw/langchain-paper.pdf")
pdf_data = loader.load();

In [16]:
pdf_chunk = text_splitter(pdf_data, 500,20 )
ids = vectordb.get()["ids"]
vectordb=Chroma.from_documents(pdf_chunk, embed_model())

In [24]:
from langchain.retrievers.multi_query import MultiQueryRetriever

query='how is langchain used for large language models'

retreiver = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(),
    llm=llm()
)

In [25]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [31]:
docs = retriever.invoke(query)
for doc in docs:
  print("*"*30)
  print(doc.page_content)

******************************
II. LANGCHAIN  
LangChain , with its open -source essence, emerges as a 
promising solution, aiming to simplify the complex process of 
developing applications powered by large language models 
(LLMs) . This framework though the rapid delivery of building 
blocks and pre -built chains for building large language model 
applications shows the easy way developers can do it .
******************************
LangChain provides a lot of utilities for adding memory to a system. These utilities can be used by themselves or 
incorporated seamlessly into a chain.  
A memory system must support two fundamental 
actions: reading and writing. Remember that each chain has 
some fundamental execution mechanism that requires 
specific inputs. Some of these inputs are provided directly by 
the user, while others may be retrieve d from memory. In a
******************************
chain of prompts that the large language model (in this 
case, GPT -4) will process.  
Step 6. 